In [1]:
import cv2
import mediapipe as mp
import os

In [21]:
def initialize_mediapipe():
    mp_hands = mp.solutions.hands
    mp_drawing = mp.solutions.drawing_utils
    hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5, min_tracking_confidence=0.5)
    return mp_hands, mp_drawing, hands

def procesar_frame(frame, hands):
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = hands.process(image)
    return results

def calcular_diferencia(puntos_previos,puntos_actuales ):
    if puntos_previos is None or puntos_actuales is None:
        return 10.0
    
    # Comprobamos si alguna de las listas está vacía o si las dos listas tienen diferentes longitudes
    if len(puntos_previos) != len(puntos_actuales):
        return 10.0

    num_hands_previas = len(puntos_previos)
    num_hands_actuales = len(puntos_actuales)
    print("num manos previas", num_hands_previas)

    if num_hands_previas != num_hands_actuales:
        return 10.0

    total_diff = 0.0

    for i in range(num_hands_previas):
        hand_prev = puntos_previos[i].landmark
        hand_actual = puntos_actuales[i].landmark

        diff = sum([
            abs(lm1.x - lm2.x) + abs(lm1.y - lm2.y) + abs(lm1.z - lm2.z)
            for lm1, lm2 in zip(hand_prev, hand_actual)
        ])
        total_diff += diff
    print("Total dif", total_diff)
    if num_hands_previas >= 2:
        print(total_diff)
        total_diff /= num_hands_previas
    print(total_diff)
    return float(total_diff)

def extract_key_frames(video, threshold=4.4, min_frame_interval=5):
    mp_hands, mp_drawing, hands = initialize_mediapipe()
    puntos_previos = []
    key_frames = []
    frame_count = 0
    print("Iniciando la extracción de frames clave")

    while video.isOpened():
        ret, frame = video.read()
        if not ret:
            print("Fin del video o error al leer frame")
            break
        
        frame_count += 1
        if len(key_frames)==0 or frame_count - key_frames[-1][1] > min_frame_interval:
            results = procesar_frame(frame, hands)
            
            if results.multi_hand_landmarks:
                puntos_actuales = results.multi_hand_landmarks
                diff = calcular_diferencia(puntos_previos, puntos_actuales)

                if diff > threshold :
                    key_frames.append((frame, frame_count))
                    puntos_previos = puntos_actuales
            else:
                puntos_previos = None

    video.release()
    return [frame for frame, _ in key_frames]

def save_key_frames(key_frames, output_folder):
    for idx, key_frame in enumerate(key_frames):
        cv2.imwrite(f'{output_folder}/key_frame_{idx}.png', key_frame)

In [22]:
video_path = 'C:\\Users\\48113164\\Documents\\GitHub\\SignAI-IA.dev\\recursos\\videoprueba1.mp4'
output_folder='C:\\Users\\48113164\\Documents\\GitHub\\SignAI-IA.dev\\recursos\\frames'
video = cv2.VideoCapture(video_path)
if not os.path.isfile(video_path):
    print(f"Error: El archivo {video_path} no existe.")
elif not video.isOpened():
    print("Error: No se pudo abrir el video.")
else:
    key_frames = extract_key_frames(video)
    save_key_frames(key_frames, output_folder)

Iniciando la extracción de frames clave


c:\Users\maxis\OneDrive\Documentos\GitHub\SignAI-IA.dev\.venv\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


num manos previas 2
Total dif 9.073470835085601
9.073470835085601
4.5367354175428005
num manos previas 2
Total dif 4.427069073839618
4.427069073839618
2.213534536919809
num manos previas 2
Total dif 6.2241702218711765
6.2241702218711765
3.1120851109355883
num manos previas 2
Total dif 6.172707768359473
6.172707768359473
3.0863538841797364
num manos previas 2
Total dif 6.218062195822654
6.218062195822654
3.109031097911327
num manos previas 2
Total dif 6.1595483203039265
6.1595483203039265
3.0797741601519633
num manos previas 2
Total dif 5.866420849387396
5.866420849387396
2.933210424693698
num manos previas 2
Total dif 5.568181271097757
5.568181271097757
2.7840906355488784
num manos previas 2
Total dif 5.39751450385485
5.39751450385485
2.698757251927425
num manos previas 2
Total dif 6.250203736170647
6.250203736170647
3.1251018680853235
num manos previas 2
Total dif 7.834119912629035
7.834119912629035
3.9170599563145174
num manos previas 2
Total dif 9.51148479758676
9.51148479758676
4.7